In [1]:
import pandas as pd
import re

In [2]:
file_path = '../data/final_training_data_3class.csv'
df = pd.read_csv(file_path)
print("original data info:")
df.info()

print("\n data preview:")
df.head()

original data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99960 entries, 0 to 99959
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sentiment   99960 non-null  int64 
 1   final_text  99960 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.5+ MB

 data preview:


,sentiment,final_text
0,2,如果我每听一次这张CD就有一美元，每回让艾丽卡（Alexa）播放一次就有一美元的话，我现在就...
1,2,“出色的音效——迫不及待想亲眼看到他们了——当他们在城里时，我总是想念他们！”
2,2,这是一张很棒的CD。音乐很好，播放也很顺畅。卖家回复非常快，三天内就收到了。
3,0,“这些不是真正的德国歌手，他们有口音。这和他们宣传的完全不一样。音乐太差了。”
4,1,我第一次是在名古屋的一家商店里听到这首歌的，立刻爱上了凯莎（Ke$ha）的《Tik Tok》...


In [4]:
def contain_chinese(text):
    """
    search for Chinese characters in the file.
    """
    # [\u4e00-\u9fa5] is the range for chinese characters
    return bool(re.search(r'[\u4e00-\u9fa5]', str(text)))

translated = df['final_text'].apply(contain_chinese)
df_cleaned = df[translated].copy()

print(f"Original data size: {len(df)}")
print(f"Preserved data size: {len(df_cleaned)}")
print(f"Cleaned data size: {len(df) - len(df_cleaned)}")

Original data size: 99960
Preserved data size: 17958
Cleaned data size: 82002


In [7]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    
    # remove HTML labels
    text = re.sub(r'<.*?>','',text)

    # remove websites
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
                  
    # remove redundant spacings, and etc into spacings
    text = re.sub(r'\s+', ' ', text).strip()

    return text


df_cleaned['cleaned_text'] = df_cleaned['final_text'].apply(clean_text)

print(df_cleaned[['final_text', 'cleaned_text']].head())

                                          final_text  \
0  如果我每听一次这张CD就有一美元，每回让艾丽卡（Alexa）播放一次就有一美元的话，我现在就...   
1            “出色的音效——迫不及待想亲眼看到他们了——当他们在城里时，我总是想念他们！”   
2             这是一张很棒的CD。音乐很好，播放也很顺畅。卖家回复非常快，三天内就收到了。   
3            “这些不是真正的德国歌手，他们有口音。这和他们宣传的完全不一样。音乐太差了。”   
4  我第一次是在名古屋的一家商店里听到这首歌的，立刻爱上了凯莎（Ke$ha）的《Tik Tok》...   

                                        cleaned_text  
0  如果我每听一次这张CD就有一美元，每回让艾丽卡（Alexa）播放一次就有一美元的话，我现在就...  
1            “出色的音效——迫不及待想亲眼看到他们了——当他们在城里时，我总是想念他们！”  
2             这是一张很棒的CD。音乐很好，播放也很顺畅。卖家回复非常快，三天内就收到了。  
3            “这些不是真正的德国歌手，他们有口音。这和他们宣传的完全不一样。音乐太差了。”  
4  我第一次是在名古屋的一家商店里听到这首歌的，立刻爱上了凯莎（Ke$ha）的《Tik Tok》...  


In [24]:
#  Load the complete data 
source_file = '../data/reviews_complete_and_cleaned.csv'
# We will create a brand new file, which is the final version for training
final_output_file = '../data/final_training_data.csv'

print(f"loading: {source_file}")
df = pd.read_csv(source_file)

#  replace all newline characters and extra spaces with a single space.
def fix_layout(text):
    if not isinstance(text, str):
        return ""
    # Replace newline characters with a space, then replace multiple spaces with a single space
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply this cleaning function to the 'translated_text' column and create a new, final clean column
df['final_text'] = df['translated_text'].apply(fix_layout)

final_df = df[['rating', 'final_text']]

# Save dataset for training
print(f"saving data to path: {final_output_file}")
final_df.to_csv(final_output_file, index=False, encoding='utf-8-sig')

print(final_df.head())

# Check for any null values
print("\n check null:")
print(final_df.isnull().sum())

loading: ../data/reviews_complete_and_cleaned.csv
saving data to path: ../data/final_training_data.csv
   rating                                         final_text
0     5.0  如果我每听一次这张CD就有一美元，每回让艾丽卡（Alexa）播放一次就有一美元的话，我现在就...
1     5.0            “出色的音效——迫不及待想亲眼看到他们了——当他们在城里时，我总是想念他们！”
2     5.0             这是一张很棒的CD。音乐很好，播放也很顺畅。卖家回复非常快，三天内就收到了。
3     1.0            “这些不是真正的德国歌手，他们有口音。这和他们宣传的完全不一样。音乐太差了。”
4     3.0  我第一次是在名古屋的一家商店里听到这首歌的，立刻爱上了凯莎（Ke$ha）的《Tik Tok》...

 check null:
rating        0
final_text    0
dtype: int64


In [2]:
import pandas as pd
# 1. 加载我们之前精加工好的、用于5分类的数据
source_file = '../data/final_training_data.csv'
# 2. 定义我们新的、用于3分类的最终数据集的文件名
output_file_3class = '../data/final_training_data_3class.csv'

print(f"Loading data from: {source_file}")
df = pd.read_csv(source_file)

# 3. 定义标签映射函数
def map_rating_to_sentiment(rating):
    if rating in [1, 2]:
        return 0  # Negative
    elif rating == 3:
        return 1  # Neutral
    elif rating in [4, 5]:
        return 2  # Positive
    else:
        return None # Handle other cases if any

# 4. 创建新的'sentiment'列
print("Creating new 3-class sentiment labels...")
df['sentiment'] = df['rating'].apply(map_rating_to_sentiment)

# 5. 筛选出我们最终需要的列，并丢弃有问题的行
final_df = df[['sentiment', 'final_text']].dropna()
# 确保sentiment是整数类型
final_df['sentiment'] = final_df['sentiment'].astype(int)

# 6. 保存这个全新的、用于3分类训练的数据集
print(f"Saving 3-class data to: {output_file_3class}")
final_df.to_csv(output_file_3class, index=False, encoding='utf-8-sig')

print("\n--- 3-Class dataset creation complete! ---")
print("New dataset preview:")
print(final_df.head())
print("\nNew class distribution:")
print(final_df['sentiment'].value_counts())

Loading data from: ../data/final_training_data.csv
Creating new 3-class sentiment labels...
Saving 3-class data to: ../data/final_training_data_3class.csv

--- 3-Class dataset creation complete! ---
New dataset preview:
   sentiment                                         final_text
0          2  如果我每听一次这张CD就有一美元，每回让艾丽卡（Alexa）播放一次就有一美元的话，我现在就...
1          2            “出色的音效——迫不及待想亲眼看到他们了——当他们在城里时，我总是想念他们！”
2          2             这是一张很棒的CD。音乐很好，播放也很顺畅。卖家回复非常快，三天内就收到了。
3          0            “这些不是真正的德国歌手，他们有口音。这和他们宣传的完全不一样。音乐太差了。”
4          1  我第一次是在名古屋的一家商店里听到这首歌的，立刻爱上了凯莎（Ke$ha）的《Tik Tok》...

New class distribution:
sentiment
2    88112
0     6842
1     5006
Name: count, dtype: int64
